In [ ]:
%%writefile main.cpp
#include <iostream>
int main(){
  int a;
  int b = a;
  std::cout << a << "\t" << b;
}

In [ ]:
!g++ -std=c++20 main.cpp  -o demo && ./demo

In [ ]:
%%writefile Config.h
#ifndef CONFIG_H
#define CONFIG_H

#include <string>

// Declaration: This tells the compiler the variable exists and is an int,
// but does NOT allocate memory for it here.
extern const int MAX_USERS;

// Another extern example: a non-constant string
extern std::string APP_VERSION;

#endif // CONFIG_H

Overwriting Config.h


In [ ]:
%%writefile Config.cpp
#include "Config.h"
// Definition: Memory is allocated here.
// The 'extern' keyword is NOT used here because this is the definition site.
const int MAX_USERS = 500;

// Definition for the string variable
std::string APP_VERSION = "v1.2.3";

// Side effects: None

Overwriting Config.cpp


In [ ]:
%%writefile main.cpp
#include <iostream>
#include <string>
#include "Config.h"

// Side effects: Details as applicable (outputs to standard output)
int main() {
    // We can access MAX_USERS and APP_VERSION directly because they
    // were declared as extern in Config.h

    std::cout << "--- Application Configuration ---" << std::endl;
    std::cout << "Version: " << APP_VERSION << std::endl;
    std::cout << "System Capacity: " << MAX_USERS << " users" << std::endl;

    // Example usage
    int current_users = 350;
    if (current_users > MAX_USERS) {
        std::cout << "WARNING: User capacity exceeded!" << std::endl;
    } else {
        std::cout << "Current load: "
                  << (static_cast<double>(current_users) / MAX_USERS) * 100
                  << "%" << std::endl;
    }

    // Since APP_VERSION is not const, we can modify it here (if needed, though generally discouraged)
    APP_VERSION = "v1.2.4 (Hotfix)";
    std::cout << "Updated Version: " << APP_VERSION << std::endl;

    return 0;
}

Overwriting main.cpp


In [ ]:
# Compile both source files and link them into a single executable
!g++ main.cpp Config.cpp -o demo -std=c++17

# Execute the compiled program
!./demo

--- Application Configuration ---
Version: v1.2.3
System Capacity: 500 users
Current load: 70%
Updated Version: v1.2.4 (Hotfix)


### 📝 Constants.h: External Constant Declaration

This header file uses `extern const` to declare a constant variable that is initialized at runtime in another file (`Constants.cpp`). This ensures the constant is shared across the entire program without duplicating the variable definition.

In [ ]:
%%writefile Constants.h
#ifndef CONSTANTS_H
#define CONSTANTS_H

// DECLARATION: This tells the compiler the variable exists and is a constant,
// but does NOT allocate memory for it here.
extern const int MAX_MEM_MB;

#endif // CONSTANTS_H

Writing Constants.h


### 🛠️ Constants.cpp: Single Definition (Runtime Initializer)

This file contains the *one and only definition* for the `MAX_MEM_MB` constant. It uses a function (`getSystemMemorySize`) to simulate a **runtime calculation** for the initial value, proving that the constant is not compile-time.

In [ ]:
%%writefile Constants.cpp
#include "Constants.h"
#include <iostream>

// Simulated function that runs at startup to determine the const value
int getSystemMemorySize() {
    // This value is determined at runtime (e.g., querying system configuration)
    std::cout << "Calculating max memory at runtime..." << std::endl;
    return 8192; // 8 GB, just as an example value
}

// DEFINITION: 'extern const' used with a RUNTIME initializer.
// This is the single place memory is allocated for MAX_MEM_MB.
extern const int MAX_MEM_MB = getSystemMemorySize() / 2; // Allocate half of system memory

// Side effects: Details as applicable (outputs to standard output)

Writing Constants.cpp


### 🚀 Main.cpp: Application Logic and Usage

This file contains the main program entry point. It includes `Constants.h` to access the shared, runtime-initialized constant (`MAX_MEM_MB`) for a resource check, demonstrating its use across multiple files.

In [ ]:
%%writefile main.cpp
#include <iostream>
#include "Constants.h" // Includes the extern declaration

// Side effects: Details as applicable (outputs to standard output)
int main() {
    std::cout << "Application started." << std::endl;

    // Use the shared, runtime-initialized const variable
    int current_usage = 1500;

    std::cout << "--- Resource Check ---" << std::endl;
    std::cout << "Current usage: " << current_usage << " MB" << std::endl;
    std::cout << "Shared Max Memory: " << MAX_MEM_MB << " MB" << std::endl;

    if (current_usage > MAX_MEM_MB) {
        std::cerr << "ERROR: Memory limit exceeded!" << std::endl;
    } else {
        std::cout << "Remaining capacity: " << MAX_MEM_MB - current_usage << " MB" << std::endl;
    }

    return 0;
}

Overwriting main.cpp


In [ ]:
# 1. Compile both source files and link them into a single executable
!g++ -o app main.cpp Constants.cpp -std=c++17

# 2. Execute the application
!./app

Calculating max memory at runtime...
Application started.
--- Resource Check ---
Current usage: 1500 MB
Shared Max Memory: 4096 MB
Remaining capacity: 2596 MB


### 💡 Scenario: Local Const Definitions (No Linker Error)

When a `const` variable is initialized by a **constant expression** (a literal value like `100`), C++ gives it **internal linkage by default**.

This means each `.cpp` file that includes the constant header gets its own *independent, local copy* of the variable. Because these copies are local, they do not conflict during the final linking stage.

In [ ]:
%%writefile Constants.h
#ifndef CONSTANTS_H
#define CONSTANTS_H

// DEFINITION: No 'extern' and initialized with a constant expression (20).
// Each file that includes this header will get its own private copy of this const variable.
const int BUFFER_SIZE = 20;

#endif // CONSTANTS_H

Overwriting Constants.h


ModuleA.cpp (First Usage and Definition)
This file defines the constant locally when it includes Constants.h.

In [ ]:
%%writefile ModuleA.cpp
#include <iostream>
#include "Constants.h"

// Function in Module A uses the constant
void use_buffer_a() {
    std::cout << "Module A: Using buffer size " << BUFFER_SIZE << std::endl;
}

// Side effects: Details as applicable (outputs to standard output)

Writing ModuleA.cpp


ModuleB.cpp (Second Usage and Definition)
This file defines the constant locally again when it includes Constants.h.

In [ ]:
%%writefile ModuleB.cpp
#include <iostream>
#include "Constants.h"

// Function in Module B uses the constant
void use_buffer_b() {
    std::cout << "Module B: Using buffer size " << BUFFER_SIZE << std::endl;
}

// Side effects: Details as applicable (outputs to standard output)

Writing ModuleB.cpp


Main.cpp (Coordination and Execution)
This file coordinates the execution of the functions defined in the other modules.

In [ ]:
%%writefile Main.cpp
// Function declarations (defined in separate files)
void use_buffer_a();
void use_buffer_b();

// Side effects: Details as applicable (outputs to standard output)
int main() {
    use_buffer_a();
    use_buffer_b();
    // Program links successfully despite BUFFER_SIZE being defined in ModuleA.cpp and ModuleB.cpp
    return 0;
}

Overwriting Main.cpp


Linking and Execution

In [ ]:
# 1. Compile all source files. The linker successfully combines them.
!g++ -o app ModuleA.cpp ModuleB.cpp Main.cpp -std=c++17

# 2. Execute the application
!./app

Module A: Using buffer size 20
Module B: Using buffer size 20


In [ ]:
%%writefile scope_example.cpp
#include <iostream>

// Program for illustration purposes only: It is bad style for a function
// to use a global variable and also define a local variable with the same name
int reused = 42; // reused has global scope

int main()
{
    int unique = 0; // unique has block scope
    // output #1: uses global reused; prints 42 0
    std::cout << reused << " " << unique << std::endl;
    int reused = 0; // new, local object named reused hides global reused
    // output #2: uses local reused; prints 0 0
    std::cout << reused << " " << unique << std::endl;
    // output #3: explicitly requests the global reused; prints 42 0
    std::cout << ::reused << " " << unique << std::endl;
    return 0;
}

In [ ]:
%%writefile simple_scope.cpp
#include <iostream>
int i = 42;
int main()
{
    int i = 100;
    int j = i;
    std::cout << "j = " << j;
    return 0;
}

In [ ]:
%%writefile loop_scope_example.cpp
#include <iostream>

int main() {
    int i = 100, sum = 0;
    for (int i = 0; i != 10; ++i)
        sum += i;
    std::cout << i << " " << sum << std::endl;
    return 0;
}

In [ ]:
# Compile the C++ code
!g++ loop_scope_example.cpp -o loop_scope_demo

# Run the compiled executable
!./loop_scope_demo

In [ ]:
%%writefile reference_declarations.cpp
#include <iostream>

int main() {
    int i = 1024, i2 = 2048; // i and i2 are both ints
    int &r = i, r2 = i2; // r is a reference bound to i; r2 is an int
    int i3 = 1024, &ri = i3; // i3 is an int; ri is a reference bound to i3
    int &r3 = i3, &r4 = i2; // both r3 and r4 are references

    // You can add some output here to see the values and confirm
    // the references are bound correctly.
    std::cout << "i: " << i << ", r: " << r << ", i2: " << i2 << ", r2: " << r2 << std::endl;
    std::cout << "i3: " << i3 << ", ri: " << ri << ", r3: " << r3 << ", r4: " << r4 << std::endl;

    i3 = 2048;
    std::cout << "i3: " << i3 << ", ri: " << ri << ", r3: " << r3 << std::endl;

    return 0;
}

Writing reference_declarations.cpp


In [ ]:
# Compile the C++ code
!g++ reference_declarations.cpp -o reference_demo

# Run the compiled executable
!./reference_demo

i: 1024, r: 1024, i2: 2048, r2: 2048
i3: 1024, ri: 1024, r3: 1024, r4: 2048
i3: 2048, ri: 2048, r3: 2048


In this example, we use references in two ways:

1.  **Binding a reference to a specific element:** `int& first_element = numbers[0];` creates a reference `first_element` that is an alias for the first element of the `numbers` vector. Any changes made to `first_element` directly affect `numbers[0]`. This is more efficient than copying the element if you need to modify it repeatedly.

2.  **Using references in a range-based for loop:** `for (int& num : numbers)` uses a reference `num` for each element in the `numbers` vector. This allows you to modify the elements of the vector directly within the loop without creating temporary copies. If you were to use `for (int num : numbers)`, `num` would be a copy of each element, and modifying `num` would not affect the original vector.

This demonstrates how references can be used to work with data more efficiently, especially when dealing with larger data structures or objects, by avoiding unnecessary copying.

In [ ]:
# Compile the C++ code
!g++ practical_references.cpp -o practical_references_demo

# Run the compiled executable
!./practical_references_demo

Original vector:
1 5 2 8 3 9 4 
Maximum element found: 9
Vector after modifying the maximum element using a reference:
1 5 2 8 3 -1 4 
Vector after doubling each element using references:
2 10 4 16 6 -2 8 


In [ ]:
%%writefile function_references.cpp
#include <iostream>
#include <vector>
#include <numeric> // For std::accumulate

// Function that takes a vector by value (creates a copy)
void process_vector_by_value(std::vector<int> vec) {
    std::cout << "Inside process_vector_by_value (copy):" << std::endl;
    // Modifying 'vec' here only affects the local copy
    for (int& num : vec) {
        num *= 10;
    }
    // The original vector outside this function remains unchanged
    long long sum = std::accumulate(vec.begin(), vec.end(), 0LL);
    std::cout << "Sum of elements in local copy: " << sum << std::endl;
}

// Function that takes a vector by reference (no copy)
void process_vector_by_reference(std::vector<int>& vec) {
    std::cout << "Inside process_vector_by_reference (original):" << std::endl;
    // Modifying 'vec' here modifies the original vector
    for (int& num : vec) {
        num *= 10;
    }
     long long sum = std::accumulate(vec.begin(), vec.end(), 0LL);
    std::cout << "Sum of elements in original vector: " << sum << std::endl;
}

int main() {
    std::vector<int> my_vector = {1, 2, 3, 4, 5};

    std::cout << "Original vector:" << std::endl;
    for (int number : my_vector) {
        std::cout << number << " ";
    }
    std::cout << std::endl;

    // Calling the function that takes by value
    process_vector_by_value(my_vector);

    std::cout << "Vector after calling process_vector_by_value (should be unchanged):" << std::endl;
    for (int number : my_vector) {
        std::cout << number << " ";
    }
    std::cout << std::endl;

    // Calling the function that takes by reference
    process_vector_by_reference(my_vector);

    std::cout << "Vector after calling process_vector_by_reference (should be modified):" << std::endl;
    for (int number : my_vector) {
        std::cout << number << " ";
    }
    std::cout << std::endl;

    return 0;
}

Overwriting function_references.cpp


In [ ]:
# Compile the C++ code
!g++ function_references.cpp -o function_references_demo

# Run the compiled executable
!./function_references_demo

Original vector:
1 2 3 4 5 
Inside process_vector_by_value (copy):
Sum of elements in local copy: 150
Vector after calling process_vector_by_value (should be unchanged):
1 2 3 4 5 
Inside process_vector_by_reference (original):
Sum of elements in original vector: 150
Vector after calling process_vector_by_reference (should be modified):
10 20 30 40 50 


This example demonstrates a key practical use case for references: passing objects to functions.

1.  **`process_vector_by_value(std::vector<int> vec)`:** This function takes a `std::vector<int>` by value. When you pass a vector by value, a **complete copy** of the original vector is created inside the function. Any modifications made to `vec` within this function only affect this local copy. The original `my_vector` in `main` remains unchanged. This can be inefficient, especially for large vectors, due to the time and memory required for copying.

2.  **`process_vector_by_reference(std::vector<int>& vec)`:** This function takes a `std::vector<int>` by reference (`&`). When you pass a vector by reference, **no copy** is made. The `vec` inside the function is an alias for the original `my_vector` in `main`. Any modifications made to `vec` within this function directly affect the original `my_vector`. This is much more efficient for larger objects as it avoids the copying overhead.

**When to use pass by reference:**

*   When you need the function to modify the original object that was passed in.
*   When passing large objects to a function to improve performance and reduce memory usage by avoiding unnecessary copying.

This is a very common pattern in C++ for working with objects efficiently within functions.

#Pointers

| Declaration | Type | Relationship |
| :--- | :--- | :--- |
| `int i = 42;` | `int` | The primary object, holding the value $\text{42}$. |
| `int &r = i;` | `int&` | `r` is an alias for i. |
| `int *p = &i;` | `int*` | `p` is a pointer to the memory address of i. |
| `int &r2 = *p;` | `int&` | `r2` is an alias for the object that p points to (which is i). |

## 📋 C++ Null Pointer Initialization Methods

| Declaration | Needs `<cstdlib>`? | Rationale and Modern Status |
| :--- | :--- | :--- |
| **`int *p1 = nullptr;`** | **No** | **Recommended Modern Practice (C++11+).** `nullptr` is a **keyword** built into the language; no header is required. It is the type-safe solution. |
| **`int *p2 = 0;`** | **No** | **Technically, No.** The integer literal `0` can be implicitly converted to a null pointer value, a capability inherent to the language. |
| **`int *p3 = NULL;`** | **Yes** | **Discouraged Old Practice.** `NULL` is a **preprocessor macro** and requires a standard header like `<cstdlib>` for its definition. |

###Null Pointer

## 📋 Advice: Initialize all Pointers; C++ Null Pointer Initialization Methods

| Declaration | Needs `<cstdlib>`? | Rationale and Modern Status |
| :--- | :--- | :--- |
| **`int *p1 = nullptr;`** | **No** | **Recommended Modern Practice (C++11+).** `nullptr` is a **keyword** built into the language; no header is required. It is the type-safe solution. |
| **`int *p2 = 0;`** | **No** | **Technically, No.** The integer literal `0` can be implicitly converted to a null pointer value, a capability inherent to the language. |
| **`int *p3 = NULL;`** | **Yes** | **Discouraged Old Practice.** `NULL` is a **preprocessor macro** and requires a standard header like `<cstdlib>` for its definition. |


# 📋 C++ Pointers and Variables: Declarations vs. Assignments (Minimal Interactive)

| Code Line | Operation and Explanation |
| :--- | :--- |
| `int i = 42;` | <details><summary>▶️</summary>**Type/Operation:** Definition<br>**Explanation:** `i` is defined and initialized as a normal integer object with the value **42**.</details> |
| `int *pi = 0;` | <details><summary>▶️</summary>**Type/Operation:** Definition<br>**Explanation:** `pi` is defined as a pointer to `int`. It is initialized to the **null pointer value** (equivalent to `nullptr`). It addresses no object.</details> |
| `int *pi2 = &i;` | <details><summary>▶️</summary>**Type/Operation:** Definition<br>**Explanation:** `pi2` is defined as a pointer to `int`. It is initialized to hold the **address of** `i` (`&i`).</details> |
| `int *pi3;` | <details><summary>▶️</summary>**Type/Operation:** Declaration<br>**Explanation:** `pi3` is declared as a pointer to `int`. Since it's local, its value (the address it holds) is **uninitialized (garbage)**.</details> |
| `pi3 = pi2;` | <details><summary>▶️</summary>**Type/Operation:** Assignment<br>**Explanation:** The **value** stored in `pi2` (the address of `i`) is copied into `pi3`. Now, both `pi2` and `pi3` point to the same object (`i`).</details> |
| `pi2 = 0;` | <details><summary>▶️</summary>**Type/Operation:** Assignment<br>**Explanation:** The value of `pi2` is changed to the **null pointer value**. `pi2` no longer points to `i`. `pi3` is unaffected and still points to `i`.</details> |
| `pi = &i;` | <details><summary>▶️</summary>**Type/Operation:** Assignment<br>**Explanation:** The value of `pi` (which was null) is changed to the **address of** `i`. `pi` now points to `i`.</details> |
| `*pi = 0;` | <details><summary>▶️</summary>**Type/Operation:** Dereference/Assignment<br>**Explanation:** The **value** of the object *pointed to by* `pi` (which is `i`) is changed to **0**. The pointer `pi` itself remains unchanged; it still points to the same address.</details> |

## 📋 Pointer Initialization and Boolean Evaluation

| Code Line | Type/Operation | Value Stored in Variable | Boolean Evaluation (`if`) |
| :--- | :--- | :--- | :--- |
| `int ival = 1024;` | **Definition** (`int`) | The memory location holds the integer value $\text{1024}$. | N/A |
| `int *pi = 0;` | **Definition** (`int*`) | The pointer `pi` holds the **null pointer value** ($\text{0}$ or `nullptr`). | **`if (pi)` is FALSE.** (A null pointer evaluates to `false`). |
| `int *pi2 = &ival;` | **Definition** (`int*`) | The pointer `pi2` holds the **memory address** of `ival` (a non-zero address). | N/A |
| `if (pi)` | **Conditional Check** | N/A | **`false`**. (Because `pi` is null). |
| `if (pi2)` | **Conditional Check** | N/A | **`true`**. (Because `pi2` holds a valid, non-zero memory address). |

Pointers VS References

| Feature | Pointer (`int *p`) | Reference (`int &r`) |
| :--- | :--- | :--- |
| **Binding** | Can be left uninitialized (holds garbage) or initialized to `nullptr`. | **Must be initialized** to an object at the time of declaration. |
| **Reassignment**| Can be **reassigned** at any time to point to a different object. | **Cannot be reassigned.** It is permanently bound to the object it was initialized with (it is an alias). |
| **Indirection** | Requires explicit dereferencing (`*p`) to access the object's value. | Accesses the object's value **implicitly** (no `*` needed). |
| **Null State** | Can be set to **`nullptr`** (null) to indicate it points to nothing valid. | **Cannot be null.** Must always refer to a valid object. |
| **Address** | Has its own memory address. You can take the address of a pointer (`&p`). | Does not have its own distinct address (it's conceptual). Taking its address (`&r`) returns the address of the object it aliases. |

## 📋 Code Analysis
```cpp
int i = 42;
int *p1 = &i;
*p1 = *p1 * *p1;

# 📋 Exercise 2.21: Legality Check, Definitions and Legality (Given `int i = 0;`)

| Statement | Answer (Click to Reveal) |
| :--- | :--- |
| (a) `double* dp = &i;` | <details><summary>Click for Explanation</summary>**Illegal.** A pointer must be initialized with the address of an object of the **same type**. Cannot implicitly convert `int*` to `double*`.</details> |
| (b) `int *ip = i;` | <details><summary>Click for Explanation</summary>**Illegal.** A pointer must be initialized with an **address** (or `nullptr`). `i` is an ordinary integer variable, not a literal `0` or an address.</details> |
| (c) `int *p = &i;` | <details><summary>Click for Explanation</summary>**Legal.** The pointer `p` (pointer to `int`) is correctly initialized with the **address of** (`&`) an `int` object (`i`).</details> |

## 📋 Exercise 2.23: Pointer Validity Check

| Question/Prompt | Answer (Click to Reveal) |
| :--- | :--- |
| **Ex 2.23:** Given a pointer `p`, can you determine whether `p` points to a valid object? | <details><summary>Click for Explanation</summary>**No, you generally cannot.** The expression `if (p)` only tells you if the pointer is **not null** (i.e., it holds a non-zero address). It cannot tell you if the memory at that non-zero address is:<br>1. **Validly allocated** by your program (it could be a **dangling pointer** pointing to deallocated memory).<br>2. **Accessible** (it could point to memory reserved by the operating system or another process).<br>Any attempt to read from or write to an invalid, non-null address results in **Undefined Behavior** (typically a program crash).<br></details> |

Here is the raw Markdown content for your Colab notebook, explaining the C++ two-step memory allocation process, ready to be copied into a **Text cell**.

***

## ⚙️ Memory Allocation: `malloc` and Explicit Casting

 `std::malloc` requires a two-step process in C++ to ensure type safety.

| Statement | Type | Explanation |
| :--- | :--- | :--- |
| `void* raw_memory = std::malloc(sizeof(int));` | `void*` | The `std::malloc` function returns a **generic, typeless address** (`void*`) from the heap. It simply allocates the required number of bytes. |
| `int* ptr = static_cast<int*>(raw_memory);` | `int*` | C++ is strongly typed. This step uses an **explicit cast** (`static_cast`) to convert the generic `void*` into a type-specific **`int*`**. This tells the compiler that the memory should be treated as an integer, allowing safe dereferencing later. |
```

# 📝 Exercise 2.24: Pointer Initialization Legality

| Question/Prompt | Answer (Click to Reveal) |
| :--- | :--- |
| **Ex 2.24:** Are these legal?: (i) **`void *p = &i;`** (ii) **`long *lp = &i;`** | <details><summary>**Click for Explanation**</summary>The legality is determined by C++'s rules for implicit pointer conversions:<br><br>1. **`void *p = &i;` is LEGAL:** The **`void*`** pointer is a generic pointer type designed to hold the address of *any* non-`const` data object. C++ allows an **implicit conversion** from any data pointer (`int*` in this case) to a **`void*`**.<br><br>2. **`long *lp = &i;` is ILLEGAL:** A pointer must be initialized with the address of an object of the **same type** as the pointer's base type. Here, **`lp`** is a **`long*`** (pointer to long), but **`&i`** is an **`int*`** (address of an int). Since `long` and `int` are different types, C++ **forbids this implicit conversion** to prevent accidental misalignment or incorrect dereferencing.<br></details> |

# 🚀 Modern C++ Memory Management: `new` vs. `malloc`

This table summarizes the key differences between C-style dynamic allocation (`malloc`) and the preferred modern C++ approach (`new` and Smart Pointers).

| Feature | Old Style: `std::malloc` (C-style) | Modern Style: `new` Operator (C++) | State-of-the-Art: `std::unique_ptr` |
| :--- | :--- | :--- | :--- |
| **Header** | `<cstdlib>` | None (Built-in operator) | `<memory>` |
| **Object Construction** | **No.** Allocates raw, uninitialized memory. | **Yes.** Automatically calls the object's **constructor** and allocates memory. | **Yes.** Calls constructor via `std::make_unique` or `std::make_shared`. |
| **Return Type** | **`void*`** (Typeless) | **Type-safe pointer** (e.g., `int*`) | **Smart Pointer Object** (e.g., `unique_ptr<int>`) |
| **Cleanup** | **`std::free()`** (Must be called manually; no destructor called). | **`delete`** (Must be called manually; calls destructor). | **Automatic.** Deallocation is handled by the smart pointer's destructor when it goes out of scope (RAII). |
| **Code Safety** | Low (Prone to memory leaks; requires explicit casting). | Medium (Prone to memory leaks if `delete` is forgotten). | **High** (Eliminates almost all manual memory leaks). |
| **Example** | `int* p = static_cast<int*>(malloc(sizeof(int)));` | `int* p = new int(10);` | `auto p = std::make_unique<int>(10);` |

# 📋 Pointer Declaration Styles and Scope (Interactive)

| Code Line | Explanation |
| :--- | :--- |
| `int i = 1024, *p = &i, &r = i;` | <details><summary>▶️</summary>**Multiple Declaration:** A single statement declaring three related variables:<br>1. `i`: An `int` initialized to $\text{1024}$.<br>2. `*p`: A pointer to `int` initialized with the address of `i`.<br>3. `&r`: A reference to `int` bound to `i`.</details> |
| `int* p;` | <details><summary>▶️</summary>**Stylistic Declaration:** Legally declares **`p` as a pointer to `int`**.<br>This style emphasizes that **`int*` is the pointer type**.</details> |
| `int* p1, p2;` | <details><summary>▶️</summary>**Common Pitfall:** This declares two variables in one statement:<br>1. `p1`: A pointer to `int` (because the asterisk `*` binds to the identifier `p1`).<br>2. `p2`: A plain `int` (The `*` does *not* carry over to `p2`).</details> |
| `int *p1, *p2;` | <details><summary>▶️</summary>**Correct Multiple Pointer Declaration:** Both `p1` and `p2` are correctly declared as pointers to `int`. <br>The asterisk must be explicitly repeated for every pointer declared in the list.</details> |
| `int* p1;` <br> `int* p2;` | <details><summary>▶️</summary>**Preferred Modern Style:** Declaring one variable per line avoids the "stickiness" problem entirely. <br> Ensures maximum clarity, and is the safest way to declare multiple pointers.</details> |

## 📋 Pointer-to-Pointer Initialization

| Object | Declaration | Type | Value Stored | Points To (Address of) |
| :--- | :--- | :--- | :--- | :--- |
| `ival` | `int ival = 1024;` | `int` | **1024** | N/A (The data itself) |
| `pi` | `int *pi = &ival;` | `int *` (Pointer to int) | **&ival** (Address of `ival`) | `ival` |
| `ppi` | `int **ppi = &pi;` | `int **` (Pointer to pointer to int) | **&pi** (Address of `pi`) | `pi` |

In [ ]:
%%writefile main.cpp
#include <iostream>
#include <iomanip>

/**
 * @brief Main function to demonstrate address relationships.
 * Side effects: Details as applicable (outputs to standard output)
 */
int main() {
    int ival = 1024;
    int *pi = &ival;    // pi holds the address of ival
    int **ppi = &pi;    // ppi holds the address of pi

    // Set output stream to print addresses consistently (e.g., in hex)
    std::cout << std::hex << std::showbase;

    std::cout << "\n--- Memory Relationship Demonstration ---" << std::endl;

    // 1. IVAL: The final data
    std::cout << std::left << std::setw(15) << "ival value:" << std::dec << ival << std::endl;
    std::cout << std::left << std::setw(15) << "Address of ival (&ival):" << &ival << std::endl;
    std::cout << "----------------------------------------" << std::endl;

    // 2. PI: The single pointer
    std::cout << std::left << std::setw(15) << "pi value (points to):" << pi << std::endl; // Should match &ival
    std::cout << std::left << std::setw(15) << "Address of pi (&pi):" << &pi << std::endl;
    std::cout << "Dereferenced *pi:" << std::dec << *pi << std::endl;
    std::cout << "----------------------------------------" << std::endl;

    // 3. PPI: The double pointer
    std::cout << std::left << std::setw(15) << "ppi value (points to):" << ppi << std::endl; // Should match &pi
    std::cout << "Dereferenced *ppi:" << *ppi << std::endl; // Prints the address of ival (value of pi)
    std::cout << "Doubly Dereferenced **ppi:" << std::dec << **ppi << std::endl; // Prints the value of ival

    return 0;
}

Writing main.cpp


In [ ]:
# 1. Compile the C++ source file using the C++17 standard
!g++ -o app main.cpp -std=c++17

# 2. Execute the compiled program
!./app


--- Memory Relationship Demonstration ---
ival value:    1024
Address of ival (&ival):0x7ffeecb14a24
----------------------------------------
pi value (points to):0x7ffeecb14a24
Address of pi (&pi):0x7ffeecb14a28
Dereferenced *pi:1024
----------------------------------------
ppi value (points to):0x7ffeecb14a28
Dereferenced *ppi:0x7ffeecb14a24
Doubly Dereferenced **ppi:1024


# 📋 Reference to a Pointer (`int *&r`) Operations (Interactive)

| Code Line | Explanation |
| :--- | :--- |
| `int i = 42;` | <details><summary>▶️</summary>**Definition:** The primary integer object, initialized to **42**.</details> |
| `int *p;` | <details><summary>▶️</summary>**Declaration:** A pointer to `int`. Its value is **uninitialized (garbage)**.</details> |
| `int *&r = p;` | <details><summary>▶️</summary>**Definition (Key Step):** `r` is a **reference to the pointer `p`**. Any operation on `r` directly modifies the pointer `p`.</details> |
| `r = &i;` | <details><summary>▶️</summary>**Assignment (Changes Pointer):** This is equivalent to writing **`p = &i;`**. The pointer `p` now holds the address of `i`.</details> |
| `*r = 0;` | <details><summary>▶️</summary>**Dereference/Assignment (Changes Data):** This is equivalent to **`*p = 0;`**. <br>It changes the value of the object pointed to by `p` (which is `i`) to **0**. <br> The pointer `p` itself is unchanged.</details> |

In [ ]:
%%writefile ref_to_pointer_demo.cpp
#include <iostream>
#include <iomanip>
#include <string>

// Declare/define names in the closest possible scope, not the widest feasible.
// Avoid global data as much as possible.

/**
 * @brief Prints the state of the variables i, p, and the object *p points to.
 * @param stage The description of the current operation.
 * @param i The integer variable.
 * @param p The pointer to int.
 * Side effects: Details as applicable (outputs to standard output)
 */
void print_state(const std::string& stage, int i, int* p) {
    std::cout << "\n--- " << stage << " ---" << std::endl;
    std::cout << std::left << std::setw(30) << "Value of i:" << i << std::endl;
    std::cout << std::left << std::setw(30) << "Address of i (&i):" << &i << std::endl;
    std::cout << "---------------------------------" << std::endl;
    std::cout << std::left << std::setw(30) << "Value of pointer p (address it holds):" << p << std::endl;

    // Check if the pointer is null before dereferencing
    if (p) {
        std::cout << std::left << std::setw(30) << "Value pointed to by *p:" << *p << std::endl;
    } else {
        std::cout << std::left << std::setw(30) << "Value pointed to by *p:" << "NULL POINTER (Cannot Dereference)" << std::endl;
    }
}

/**
 * @brief Main function to demonstrate reference to a pointer.
 * @return 0 on successful execution.
 * Side effects: Details as applicable (outputs to standard output)
 */
int main() {
    int i = 42;
    int *p; // p is uninitialized here (holds garbage address initially)

    // The key declaration: r is a reference to the pointer p.
    int *&r = p;

    // Initial check (p's value is garbage/uninitialized)
    std::cout << "Initial Setup: i = 42" << std::endl;
    std::cout << "Pointer p is uninitialized. DO NOT DEREFERENCE." << std::endl;

    // Operation 1: r = &i;
    // r refers to a pointer; assigning &i to r makes p point to i
    r = &i;
    print_state("1. After r = &i (p now points to i)", i, p);

    // Operation 2: *r = 0;
    // dereferencing r yields i, the object to which p points; changes i to 0
    *r = 0;
    print_state("2. After *r = 0 (i's value is changed)", i, p);

    // Final check to show r's effect
    std::cout << "\n--- Final Verification ---" << std::endl;
    std::cout << "Value of i (via i): " << i << std::endl;
    std::cout << "Value of i (via *p): " << *p << std::endl;

    return 0;
}

Writing ref_to_pointer_demo.cpp


In [ ]:
# 1. Compile the C++ source file
!g++ -o ref_ptr_app ref_to_pointer_demo.cpp -std=c++17

# 2. Execute the compiled program
!./ref_ptr_app

Initial Setup: i = 42
Pointer p is uninitialized. DO NOT DEREFERENCE.

--- 1. After r = &i (p now points to i) ---
Value of i:                   42
Address of i (&i):            0x7fff18094d94
---------------------------------
Value of pointer p (address it holds):0x7fff18094dcc
Value pointed to by *p:       42

--- 2. After *r = 0 (i's value is changed) ---
Value of i:                   0
Address of i (&i):            0x7fff18094d94
---------------------------------
Value of pointer p (address it holds):0x7fff18094dcc
Value pointed to by *p:       0

--- Final Verification ---
Value of i (via i): 0
Value of i (via *p): 0


# 📋 Exercise 2.25: Determining Types and Values (Interactive)

| Statement | Variables, Types, and Values |
| :--- | :--- |
| (a) `int* ip, &r = ip;` | <details><summary>▶️</summary>**ip:** `int*`<br>**Value:** Uninitialized (Garbage)<br>**r:** `int*&` (Reference to a pointer)<br>**Value:** Aliases the pointer `ip`</details> |
| (b) `int i, *ip = 0;` | <details><summary>▶️</summary>**i:** `int`<br>**Value:** Uninitialized (Garbage)<br>**ip:** `int*` (Pointer to int)<br>**Value:** `nullptr`</details> |
| (c) `int* ip, ip2;` | <details><summary>▶️</summary>**ip:** `int*` (Pointer to int)<br>**Value:** Uninitialized (Garbage)<br>**ip2:** `int`<br>**Value:** Uninitialized (Garbage) - *The `*` does not carry over.*</details> |

In [ ]:
%%writefile demo.cpp
#include <iostream>
#include <iomanip>
#include <string>

// Declare/define names in the closest possible scope, not the widest feasible.
// Avoid global data as much as possible.

/**
 * @brief Prints the value or address of the variable.
 * Side effects: Details as applicable (outputs to standard output)
 */
void print_variable(const std::string& name, auto value) {
    std::cout << std::left << std::setw(15) << name + ":" << value << std::endl;
}

/**
 * @brief Main function to demonstrate variable states from Exercise 2.25.
 * @return 0 on successful execution.
 * Side effects: Details as applicable (outputs to standard output)
 */
int main() {
    std::cout << "--- Exercise 2.25 Variable States ---" << std::endl;

    // Set output stream to print addresses consistently (e.g., in hex)
    std::cout << std::hex << std::showbase << std::internal;

    // --- (a) int* ip, &r = ip; ---
    std::cout << "\n(a) int* ip, &r = ip;" << std::endl;
    int* ip;         // Uninitialized pointer (garbage address)
    int*& r = ip;    // Reference to the pointer ip

    // *p is unsafe here, so we only print the pointer's address itself
    print_variable("ip (Uninit Ptr)", ip);
    print_variable("r (Ref to Ptr)", r);
    print_variable("&r (Address of Ptr)", &r); // Note: Should equal &ip

    // --- (b) int i, *ip_b = 0; ---
    std::cout << "\n(b) int i, *ip_b = 0;" << std::endl;
    int i;           // Uninitialized int (Garbage)
    int *ip_b = 0;   // Pointer initialized to null

    // We can only safely print the address of i, not its value, to avoid UB.
    // However, to see the garbage, we read it once.
    std::cout << std::dec << std::left << std::setw(15) << "i (Uninit Int):" << i << std::endl;
    std::cout << std::hex; // Switch back to hex for pointers
    print_variable("ip_b (nullptr)", ip_b);

    // --- (c) int* ip_c, ip2; ---
    std::cout << "\n(c) int* ip_c, ip2;" << std::endl;
    int* ip_c;       // Uninitialized pointer (Garbage address)
    int ip2;         // Uninitialized int (Garbage value)

    print_variable("ip_c (Uninit Ptr)", ip_c);
    std::cout << std::dec << std::left << std::setw(15) << "ip2 (Uninit Int):" << ip2 << std::endl;

    std::cout << "\n*NOTE: Values labeled 'Uninit' are unpredictable 'garbage' values." << std::endl;

    return 0;
}

Writing demo.cpp


In [ ]:
!g++ demo.cpp -o demo -std=c++20
!./demo

--- Exercise 2.25 Variable States ---

(a) int* ip, &r = ip;
ip (Uninit Ptr):0
r (Ref to Ptr):0
&r (Address of Ptr):0x7fff3d5133a0

(b) int i, *ip_b = 0;
i (Uninit Int):0
ip_b (nullptr):0

(c) int* ip_c, ip2;
ip_c (Uninit Ptr):0
ip2 (Uninit Int):0

*NOTE: Values labeled 'Uninit' are unpredictable 'garbage' values.


# 📋 C++ `const` Qualifier and Initialization (Interactive)

| Statement | Status and Explanation |
| :--- | :--- |
| `const int i = get_size();` | <details><summary>▶️</summary>**Status:** **Valid**<br>**Explanation:** `i` is a constant initialized at **runtime**. Its value is fixed throughout its lifetime.</details> |
| `const int j = 42;` | <details><summary>▶️</summary>**Status:** **Valid**<br>**Explanation:** `j` is a constant initialized at **compile time**. The compiler substitutes $\text{42}$ wherever `j` is used.</details> |
| `const int k;` | <details><summary>▶️</summary>**Status:** **Illegal (Error)**<br>**Explanation:** All `const` variables **must be initialized** at the point of definition, as their value cannot be set later.</details> |
| `const int ci = i;` | <details><summary>▶️</summary>**Status:** **Valid**<br>**Explanation:** `i`'s value ($\text{42}$) is copied into `ci`. Both are now $\text{42}$.</details> |
| `int j = ci;` | <details><summary>▶️</summary>**Status:** **Valid**<br>**Explanation:** The value of the constant `ci` ($\text{42}$) is copied into the non-constant integer `j`. The constness is not inherited.</details> |

# 📋 Binding `const` References (Interactive)

| Statement | Explanation |
| :--- | :--- |
| `int i = 42;` | <details><summary>▶️</summary>**Definition:** Creates a plain, modifiable integer object **`i`** initialized to 42.</details> |
| `const int &r1 = i;` | <details><summary>▶️</summary>**Status: Valid.**<br>**Explanation:** A **reference to const** (`const int&`) can bind to a non-const object (`i`).<br>This is safe because `r1` can read `i`'s value but cannot modify it, respecting the constness of the reference itself.</details> |
| `const int &r2 = 42;` | <details><summary>▶️</summary>**Status: Valid.**<br>**Explanation:** A **reference to const** can bind to a literal or the result of a conversion.<br>The compiler creates a temporary `int` object, and `r2` binds to it, **extending the temporary's lifetime**.</details> |
| `const int &r3 = r1 * 2;` | <details><summary>▶️</summary>**Status: Valid.**<br>**Explanation:** The expression yields a temporary `int` object ($\text{84}$).<br>A **reference to const** is allowed to bind to this temporary object, extending its lifetime.</details> |
| `int &r4 = r1 * 2;` | <details><summary>▶️</summary>**Status: Illegal (Error).**<br>**Explanation:** `r4` is a **plain, non-const reference** (`int&`).<br>A non-const reference **cannot bind to a temporary object** (like the result of `r1 * 2`), as this would lead to immediate, unstable bugs.</details> |

##Concept: "Read-Only View" (Binding const to Non-const)
In this example, we use a reference to provide a read-only view (const) of a shared, mutable (changeable) data object (inventory_count).

In [ ]:
// Assuming these are declared outside the loops (e.g., globals or constants):
// int inventory_count = 1000;
// const int daily_sales = 10;
// const int days_in_month = 30;
// const int n_months = 12;
for (int month = 1; month <= n_months; month++) {
    std::cout << "\n--- Starting Month " << month << " ---" << std::endl;

    for (int day = 1; day <= days_in_month; ++day) {

        // We bind a const reference (read-only view) to the mutable data
        // This ensures the inner report loop cannot modify the inventory.
        // The reference is re-bound to the current inventory_count address in each 'day' iteration.
        const int& daily_snapshot = inventory_count;

        // Inner loop uses the snapshot for logging/reporting
        for (int report_hour = 1; report_hour <= 24; ++report_hour) {
            // Safe read operation
            std::cout << "Day " << day << ", Hour " << report_hour
                      << ": Stock = " << daily_snapshot << std::endl;
        }

        // The mutable original object can still be changed directly.
        // Update inventory for the next day's starting value.
        inventory_count -= daily_sales;
    }
}

SyntaxError: unterminated string literal (detected at line 13) (ipython-input-3873321879.py, line 13)

# 📝 Binding `const` References: The Read-Only View

| Statement | Role / Status | Explanation |
| :--- | :--- | :--- |
| `int mutable_data = 100;` | <details><summary>▶️</summary>**Object Role:** **Source Data**<br>**Status:** Plain, modifiable object.</details> |
| `const int& read_view = mutable_data;` | <details><summary>▶️</summary>**Binding:** **Valid.**<br>**Explanation:** A **reference to const** (`const int&`) binds safely to a non-const object (`mutable_data`).<br>This creates a **read-only alias** for the mutable data.</details> |
| `read_view = 50;` | <details><summary>▶️</summary>**Status:** **Illegal (Error).**<br>**Explanation:** Even though `mutable_data` is non-const, the reference `read_view` is **const**. You cannot use the reference to modify the object.</details> |
| `mutable_data = 50;` | <details><summary>▶️</summary>**Status:** **Valid.**<br>**Explanation:** The original object is **non-const**, so it can be modified directly.<br>The value seen via `read_view` instantly changes to 50.</details> |

# 💡 Concept: Const Reference vs. Const Variable in a Loop

This concept demonstrates two ways to capture a "snapshot" of a mutable variable (`inventory_count`) inside a loop and explains why the memory usage differs.

| Initialization Style | `const int& snapshot = inventory_count;` | `const int snapshot = inventory_count;` |
| :--- | :--- | :--- |
| **Type** | **Constant Reference (`const int&`)** | **Constant Variable (`const int`)** |
| **Action on Init** | **Binds** to the memory address of the original `inventory_count`. | **Copies** the value from `inventory_count` into new memory. |
| **Memory Used** | Does **not** allocate new memory for the data; only allocates space for the reference address. | **Allocates new memory** on the stack for the `snapshot` integer itself. |
| **Behavior in Loop** | The reference is **re-bound** on every iteration of the outer loop (`day`). | The variable is **re-created** and re-initialized with a copy on every iteration of the outer loop (`day`). |
| **Benefit** | **Efficiency:** Minimal memory overhead, as no new integer object is created per loop. | **Safety:** Captures an independent value; cannot accidentally reflect any external changes. |

---

## 🚀 The Inventory Loop

Consider the inner reporting loop (running 24 times) accessing the snapshot:

```cpp
// inventory_count is modified *after* the reporting loop.

for (int day = 1; day <= days_in_month; ++day) {
    // Inventory starts the day at 1000
    // ...
    const int& ref_snapshot = inventory_count; // Binds to inventory_count's address
    const int val_snapshot = inventory_count;  // Copies 1000 into new memory
    
    // Inner Loop (Reports 24 times):
    // ... std::cout << ref_snapshot; // Reads 1000 from inventory_count's address
    // ... std::cout << val_snapshot; // Reads 1000 from val_snapshot's own memory
    
    inventory_count -= daily_sales; // inventory_count is now 990
}

# 🚀 Demonstration: `const` Reference vs. `const` Value

This example illustrates why passing a large object by **constant reference (`const T&`)** is essential for performance, even when the data cannot be modified by the function.

The code defines two functions that calculate the average of a large vector:
1.  **`_by_value`**: Forces a full, expensive **copy** of the entire vector.
2.  **`_by_reference`**: Passes only the **address** of the vector, incurring zero copy overhead.



%%writefile avg_calculator.cpp
#include <iostream>
#include <vector>
#include <numeric> // For std::accumulate
#include <chrono>  // For timing operations

// Declare/define names in the closest possible scope, not the widest feasible.
// Avoid global data as much as possible.

// Global constant to define the size of our sample data
const int DATA_SIZE = 100000;

// --- 1. Pass-by-Constant-Value (Expensive Copy) ---
// The compiler creates a complete, independent copy of the vector.
double calculate_avg_by_value(const std::vector<double> data_copy) {
    // A deep copy of 100,000 doubles occurs BEFORE this line runs.
    double sum = 0.0;
    for (double value : data_copy) {
        sum += value;
    }
    return sum / data_copy.size();
}
// Side effects: None

// --- 2. Pass-by-Constant-Reference (Efficient Address Pass) ---
// Only the address of the original vector is passed; no copy is made.
double calculate_avg_by_reference(const std::vector<double>& data_reference) {
    // Only the address is passed, making this operation instantaneous.
    double sum = 0.0;
    for (double value : data_reference) {
        sum += value;
    }
    return sum / data_reference.size();
}
// Side effects: None

/**
 * @brief Main function to run the demonstration and time the results.
 * @return 0 on successful execution.
 * Side effects: Details as applicable (outputs to standard output)
 */
int main() {
    // 1. Setup Large Data
    std::vector<double> sensor_readings(DATA_SIZE, 15.5);
    std::cout << "Data set size: " << DATA_SIZE << " elements." << std::endl;

    // 2. Measure Pass-by-Value (Problematic Case)
    auto start_val = std::chrono::high_resolution_clock::now();
    double avg_val = calculate_avg_by_value(sensor_readings);
    auto end_val = std::chrono::high_resolution_clock::now();
    
    auto duration_val = std::chrono::duration_cast<std::chrono::microseconds>(end_val - start_val).count();

    // 3. Measure Pass-by-Reference (Solution)
    auto start_ref = std::chrono::high_resolution_clock::now();
    double avg_ref = calculate_avg_by_reference(sensor_readings);
    auto end_ref = std::chrono::high_resolution_clock::now();
    
    auto duration_ref = std::chrono::duration_cast<std::chrono::microseconds>(end_ref - start_ref).count();

    // 4. Output Results
    std::cout << "\n--- Performance Comparison ---" << std::endl;
    
    std::cout << std::left << std::setw(30) << "Pass-by-Value Duration:" << duration_val << " microseconds" << std::endl;
    std::cout << std::left << std::setw(30) << "Pass-by-Reference Duration:" << duration_ref << " microseconds" << std::endl;
    
    std::cout << "\n--- Conclusion ---" << std::endl;
    std::cout << "The Pass-by-Value function was approximately "
              << (double)duration_val / duration_ref << " times slower." << std::endl;

    return 0;
}

In [ ]:
%%writefile avg_calculator.cpp
#include <iostream>
#include <vector>
#include <numeric> // For std::accumulate
#include <chrono>  // For timing operations
#include <iomanip> // For std::setw

// Declare/define names in the closest possible scope, not the widest feasible.
// Avoid global data as much as possible.

// Global constant to define the size of our sample data
const long long int DATA_SIZE = 1000000;

// --- 1. Pass-by-Constant-Value (Expensive Copy) ---
// The compiler creates a complete, independent copy of the vector.
double calculate_avg_by_value(const std::vector<double> data_copy) {
    // A deep copy of 1,000,000 doubles occurs BEFORE this line runs.
    double sum = 0.0;
    for (double value : data_copy) {
        sum += value;
    }
    return sum / data_copy.size();
}
// Side effects: None

// --- 2. Pass-by-Constant-Reference (Efficient Address Pass) ---
// Only the address of the original vector is passed; no copy is made.
double calculate_avg_by_reference(const std::vector<double>& data_reference) {
    // Only the address is passed, making this operation instantaneous.
    double sum = 0.0;
    for (double value : data_reference) {
        sum += value;
    }
    return sum / data_reference.size();
}
// Side effects: None

/**
 * @brief Main function to run the demonstration and time the results.
 * @return 0 on successful execution.
 * Side effects: Details as applicable (outputs to standard output)
 */
int main() {
    // 1. Setup Large Data
    std::vector<double> sensor_readings(DATA_SIZE, 15.5);
    std::cout << "Data set size: " << DATA_SIZE << " elements." << std::endl;

    // 2. Measure Pass-by-Value (Problematic Case)
    auto start_val = std::chrono::high_resolution_clock::now();
    double avg_val = calculate_avg_by_value(sensor_readings);
    auto end_val = std::chrono::high_resolution_clock::now();

    auto duration_val = std::chrono::duration_cast<std::chrono::microseconds>(end_val - start_val).count();

    // 3. Measure Pass-by-Reference (Solution)
    auto start_ref = std::chrono::high_resolution_clock::now();
    double avg_ref = calculate_avg_by_reference(sensor_readings);
    auto end_ref = std::chrono::high_resolution_clock::now();

    auto duration_ref = std::chrono::duration_cast<std::chrono::microseconds>(end_ref - start_ref).count();

    // 4. Output Results
    std::cout << "\n--- Performance Comparison ---" << std::endl;

    std::cout << std::left << std::setw(30) << "Pass-by-Value Duration:" << duration_val << " microseconds" << std::endl;
    std::cout << std::left << std::setw(30) << "Pass-by-Reference Duration:" << duration_ref << " microseconds" << std::endl;

    std::cout << "\n--- Conclusion ---" << std::endl;
    std::cout << "The Pass-by-Value function was approximately "
              << (double)duration_val / duration_ref << " times slower." << std::endl;

    return 0;
}

Writing avg_calculator.cpp


In [ ]:
# Compile the C++ source file
!g++ -o avg_calc avg_calculator.cpp -std=c++20

# Execute the compiled program
!./avg_calc

Data set size: 1000000 elements.

--- Performance Comparison ---
Pass-by-Value Duration:       19038 microseconds
Pass-by-Reference Duration:   13437 microseconds

--- Conclusion ---
The Pass-by-Value function was approximately 1.41683 times slower.


### 📊 Summary of Results

The output from the execution demonstrates the critical difference between the two styles:

| Style | Parameter | Performance | Safety Guarantee |
| :--- | :--- | :--- | :--- |
| **Pass-by-Value** | `const std::vector<T> data` | **High Overhead** (Slow) | Guaranteed (Copy is `const`) |
| **Pass-by-Reference** | `const std::vector<T>& data` | **Minimal Overhead** (Fast) | Guaranteed (Reference is `const`) |

**Conclusion:** In C++, when a function needs to read from a large object (like a container) but not modify it, the use of **`const T&`** (constant reference) is the mandatory practice for achieving **zero copy overhead** and efficient performance while maintaining strong data safety.

# 📋 Input Stream Consumption Demonstration

| Statements | Contents After Input | Marker Position in the Input Stream |
| :--- | :--- | :--- |
| **Initial State** | N/A | `957 34 1235\n` <br> `128 96\n` |
| `cin >> i >> j;` | `i = 957, j = 34` | `957 34` **`1235\n`** <br> `128 96\n` |
| `cin.ignore(100, '\n');` | No change | `957 34 1235\n` <br> **`128 96\n`** |
| `cin >> k;` | `k = 128` | `957 34 1235\n` <br> `128` **` 96\n`** |

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (ipython-input-1829637835.py, line 1)

In [ ]:
%%writefile input_demo.cpp
#include <iostream>
#include <sstream>
#include <string>

// Declare/define names in the closest possible scope, not the widest feasible.
// Avoid global data as much as possible.

/**
 * @brief Prints the state of the variables.
 * Side effects: Details as applicable (outputs to standard output)
 */
int main() {
    // 1. Setup a string stream to simulate the EXACT input from the table image.
    // The sequence is: "957 34 1235\n128 96\n"
    std::string input_buffer = "957 34 1235\n128 96\n";
    std::stringstream cin_sim(input_buffer);

    // Redirect std::cin to the stringstream for execution.
    // In a regular C++ environment, you would use std::cin directly.
    // In Colab, we use the stringstream to provide fixed input for testing.
    std::istream& cin_ref = cin_sim;

    int i = 0, j = 0, k = 0;

    std::cout << "--- Initial State ---" << std::endl;
    std::cout << "Input Buffer: " << input_buffer << std::endl;
    std::cout << "---------------------\n" << std::endl;

    // --- Operation 1: cin >> i >> j; ---
    // Reads 957 into i, reads 34 into j. The stream marker stops after the 34.
    cin_ref >> i >> j;

    std::cout << "1. After cin >> i >> j;" << std::endl;
    std::cout << "   i = " << i << std::endl;  // Should be 957
    std::cout << "   j = " << j << std::endl;  // Should be 34
    std::cout << "   Marker is now BEFORE ' 1235\\n128 96\\n'" << std::endl;
    std::cout << "---------------------\n" << std::endl;


    // --- Operation 2: cin.ignore(100, '\n'); ---
    // Discards up to 100 characters, or until a newline ('\n') is found, whichever comes first.
    // This discards " 1235" and the first newline.
    cin_ref.ignore(100, '\n');

    std::cout << "2. After cin.ignore(100, '\\n');" << std::endl;
    std::cout << "   Stream discards ' 1235\\n'." << std::endl;
    std::cout << "   Marker is now BEFORE '128 96\\n'" << std::endl;
    std::cout << "---------------------\n" << std::endl;


    // --- Operation 3: cin >> k; ---
    // Skips leading whitespace (which is none here, unless the stream left a space)
    // and reads the next integer, 128, into k.
    cin_ref >> k;

    std::cout << "3. After cin >> k;" << std::endl;
    std::cout << "   k = " << k << std::endl;  // Should be 128
    std::cout << "   Marker is now BEFORE ' 96\\n'" << std::endl;
    std::cout << "---------------------\n" << std::endl;

    // Final state of the stream check
    std::string remaining_input;
    std::getline(cin_ref, remaining_input, '\0'); // Read remainder until end of buffer

    std::cout << "--- Final State ---" << std::endl;
    std::cout << "i = " << i << ", j = " << j << ", k = " << k << std::endl;
    std::cout << "Remaining stream input (The marker is here): " << remaining_input << std::endl;


    return 0;
}

Writing input_demo.cpp


In [ ]:
# 1. Compile the C++ source file
!g++ -o input_app input_demo.cpp -std=c++17

# 2. Execute the compiled program
!./input_app

--- Initial State ---
Input Buffer: 957 34 1235
128 96

---------------------

1. After cin >> i >> j;
   i = 957
   j = 34
   Marker is now BEFORE ' 1235\n128 96\n'
---------------------

2. After cin.ignore(100, '\n');
   Stream discards ' 1235\n'.
   Marker is now BEFORE '128 96\n'
---------------------

3. After cin >> k;
   k = 128
   Marker is now BEFORE ' 96\n'
---------------------

--- Final State ---
i = 957, j = 34, k = 128
Remaining stream input (The marker is here):  96



In [ ]:
%%writefile recursive_sum.cpp
#include <iostream>
#include <vector>
#include <numeric>
#include <string>

// Declare/define names in the closest possible scope, not the widest feasible.
// Avoid global data as much as possible.

// Function Prototypes (for organization)
int helper(int start, std::vector<short> numbers, int acc);
int element_sum(std::vector<short> numbers);

// --- 1. The Recursive Helper Function ---
// Objective: A recursive function that takes a vector as an input and returns the sum of
//            Vector as output.
// Input: int start, vector<short> numbers, int acc
// Output: an integer output (i.e., sum of numbers).
// Side effects: None.
int helper(int start, std::vector<short> numbers, int acc) {
    // Approach: Takes the starting index and recursively calls
    //           itself to add the number at current index to accumulator acc.

    // Base Case: If start reaches the last index
    if (start == numbers.size() - 1) {
        acc = acc + numbers[start];
        return acc;
        // This is the base case.
        // Return the current sum.
    }

    // Recursive Step: If not at the last index
    // Here we are recursively calling the helper with the next index,
    // and passing the updated current sum.
    return helper(start + 1, numbers, acc + numbers[start]);
}

// --- 2. The Main Function for Public Call ---
// This is the main function that takes a vector input and gives sum of elements.
int element_sum(std::vector<short> numbers) {
    // Here we are calling helper with initial index at 0 and initial sum = 0.
    return helper(0, numbers, 0);
}

// --- 3. Main Execution ---
int main() {
    std::vector<short> data = {10, 20, 30, 40, 50};

    std::cout << "Calculating recursive sum..." << std::endl;
    int total_sum = element_sum(data);

    std::cout << "The sum of elements is: " << total_sum << std::endl; // Expected: 150

    return 0;
}

Writing recursive_sum.cpp


In [ ]:
# Compile the C++ source file
!g++ -o recursive_sum_app recursive_sum.cpp -std=c++17

# Execute the compiled program
!./recursive_sum_app

Calculating recursive sum...
The sum of elements is: 150
